# Calculate Combined Metrics

In [1]:
import json
import pickle
import sys

import pandas as pd
import sklearn.metrics

sys.path.insert(0, "/home/jupyter")
import util.log
import util.data

In [2]:
GC_CSV_FILE = "test_data_GC.csv"
PF_CSV_FILE = "test_data_PF.csv"

In [8]:
gc_df = pd.read_csv(GC_CSV_FILE)
pf_df = pd.read_csv(PF_CSV_FILE)

In [9]:
gc_df["Sort_Result"] = "gossip"
gc_df.head()

,Unnamed: 0,Article,Source,Token_Length,Predictions,Labels,Logit_real(0),Logit_fake(1),Prob_real(0),Prob_real(1),File_Name,Sort_Result
0,0,0,gossipcop,584,1,1,-2.757137,2.776392,0.003936,0.996064,gossipcop-9843277966,gossip
1,1,1,gossipcop,641,1,1,-2.756062,2.775949,0.003942,0.996058,gossipcop-7642653086,gossip
2,2,2,gossipcop,330,0,0,2.424183,-2.559280,0.993196,0.006804,gossipcop-928427,gossip
3,3,3,gossipcop,1573,0,0,2.443434,-2.575524,0.993432,0.006568,gossipcop-953132,gossip
4,4,4,gossipcop,1227,1,1,-2.613986,2.674522,0.005024,0.994976,gossipcop-2045311114,gossip


In [10]:
pf_df["Sort_Result"] = "political"
pf_df.head()

,Unnamed: 0,Article,Source,Token_Length,Predictions,Labels,Logit_real(0),Logit_fake(1),Prob_real(0),Prob_real(1),File_Name,Sort_Result
0,0,0,gossipcop,281,1,1,-3.197902,2.537999,0.003218,0.996782,gossipcop-9324384637,political
1,1,1,politifact,15506,0,0,4.058324,-4.163477,0.999731,0.000269,politifact8737,political
2,2,2,gossipcop,351,1,0,-1.782905,1.667640,0.030753,0.969247,gossipcop-889110,political
3,3,3,politifact,218,1,1,-4.291286,3.983364,0.000255,0.999745,politifact15205,political
4,4,4,politifact,145,1,1,-2.897077,2.571214,0.004201,0.995799,politifact14490,political


In [11]:
all_df = pd.concat([pf_df, gc_df])
all_df.shape

(3660, 12)

In [12]:
# Save evaluation data to csv file
all_df.to_csv("test_data_ALL.csv")

In [15]:
# Overall Metrics
precision, recall, f1, _ = sklearn.metrics.precision_recall_fscore_support(
    all_df.Labels, all_df.Predictions, average="binary")
accuracy = sklearn.metrics.accuracy_score(all_df.Labels, all_df.Predictions)

metrics = {"description": "Combined metrics for Roberta models on sorted data.",
           "eval_notes": "Evaluating both gossipcop and poltifact data",
           "model": "Ten-epoch Roberta models, one trained on gossipcop, other on politifact",
           "Date_evaluated": "30Nov2020",
           "accuracy": accuracy,
           "precision": precision,
           "recall": recall,
           "f1": f1}
metrics

{'description': 'Combined metrics for Roberta models on sorted data.',
 'eval_notes': 'Evaluating both gossipcop and poltifact data',
 'model': 'Ten-epoch Roberta models, one trained on gossipcop, other on politifact',
 'Date_evaluated': '30Nov2020',
 'accuracy': 0.8562841530054645,
 'precision': 0.7337962962962963,
 'recall': 0.6817204301075269,
 'f1': 0.7068004459308808}

In [16]:
# Save metrics to JSON text file.
with open("eval_metrics_ALL.json", "wt") as jfile:
    json.dump(metrics, jfile)

In [19]:
cm = pd.DataFrame(sklearn.metrics.confusion_matrix(all_df.Labels, all_df.Predictions))
label_titles = {0: "real", 1: "fake"}
cm.rename(index=label_titles, columns=label_titles, inplace=True)
cm

,real,fake
real,2500,230
fake,296,634
